Hago tres neuronas de salida, cada una me da una salida. Cuando la primera me da 1, las otras -1 y ahí tengo cierta clase, y así con las tres clases.
Puedo hacer también dos neuronas de salida y codificarlo con una tabla de verdad: (-1,1) es una clase, (1,-1) es otra clase y (1,1) es otra clase

Codificación Winner Takes All: le pongo 1 a la salida más grande (si tengo 0.8 y 0.1, uso la del 1). Aunque sean las dos positivas, tengo una respuesta.

Ahora el error se hace con una sumatoria porque tengo 3 componentes de error (por las 3 salidas).